In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
	temperature=0.1,	
	streaming=True,
	callbacks=[
		StreamingStdOutCallbackHandler()
	]
)

In [7]:
# few shot prompt template
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

examples = [
	{
		"question": "프랑스에 대해서 알려줘.",
		"answer": """
		제가 알고 있는 정보는 다음과 같습니다.
		수도 : 파리
		언어 : 프랑스어
	  주요 음식 : 와인과 치즈
		화폐 : 유로화
		"""
	},
	{
		"question": "이탈리아에 대해서 알려줘.",
		"answer": """
		제가 알고 있는 정보는 다음과 같습니다.
		수도 : 로마
		언어 : 이탈리아어
	  주요 음식 : 피자와 파스타
		화폐 : 유로화
		"""
	},
	{
		"question": "그리스에 대해서 알려줘.",
		"answer": """
		제가 알고 있는 정보는 다음과 같습니다.
		수도 : 아테네
		언어 : 그리스어
	  주요 음식 : Souvlaki 와 페타 치즈
		화폐 : 유로화
		"""
	},
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
	example_prompt=example_prompt,
	examples=examples,
	suffix="Human: 이 {country}에 대해서 알려줘.",
	input_variables=["country"]
)

chain = prompt | chat
chain.invoke({"country": "대한민국"})


AI:
		제가 알고 있는 정보는 다음과 같습니다.
		수도 : 서울
		언어 : 한국어
		주요 음식 : 김치, 불고기, 비빔밥
		화폐 : 대한민국 원 (KRW)

AIMessageChunk(content='AI:\n\t\t제가 알고 있는 정보는 다음과 같습니다.\n\t\t수도 : 서울\n\t\t언어 : 한국어\n\t\t주요 음식 : 김치, 불고기, 비빔밥\n\t\t화폐 : 대한민국 원 (KRW)')

In [9]:
# few shot chat message prompt template
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

examples = [
	{
		"country": "프랑스",
		"answer": """
		제가 알고 있는 정보는 다음과 같습니다.
		수도 : 파리
		언어 : 프랑스어
	  주요 음식 : 와인과 치즈
		화폐 : 유로화
		"""
	},
	{
		"country": "이탈리아",
		"answer": """
		제가 알고 있는 정보는 다음과 같습니다.
		수도 : 로마
		언어 : 이탈리아어
	  주요 음식 : 피자와 파스타
		화폐 : 유로화
		"""
	},
	{
		"country": "그리스",
		"answer": """
		제가 알고 있는 정보는 다음과 같습니다.
		수도 : 아테네
		언어 : 그리스어
	  주요 음식 : Souvlaki 와 페타 치즈
		화폐 : 유로화
		"""
	},
]

example_prompt = ChatPromptTemplate.from_messages([
	("human", "{country} 에 대해서 알려줘."),
	("ai", "{answer}"),
])

example_template = FewShotChatMessagePromptTemplate(
	example_prompt=example_prompt,
	examples=examples
) 

final_prompt = ChatPromptTemplate.from_messages([
	("system", "당신은 지리학 전문가입니다. 질문에 짧지만 명쾌한 답변을 줄 수 있습니다."),
	example_template,
	("human", "{country}에 대해서 알려줘."),
])

chain = final_prompt | chat
chain.invoke({"country": "태국"})

제가 알고 있는 정보는 다음과 같습니다.
수도: 방콕
언어: 태국어
주요 음식: 태국 카레, 생선 소스, 타이 라면
화폐: 태국 바트

AIMessageChunk(content='제가 알고 있는 정보는 다음과 같습니다.\n수도: 방콕\n언어: 태국어\n주요 음식: 태국 카레, 생선 소스, 타이 라면\n화폐: 태국 바트')

In [13]:
# length based example selector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

examples = [
	{
		"question": "프랑스에 대해서 알려줘.",
		"answer": """
		제가 알고 있는 정보는 다음과 같습니다.
		수도 : 파리
		언어 : 프랑스어
	  주요 음식 : 와인과 치즈
		화폐 : 유로화
		"""
	},
	{
		"question": "이탈리아에 대해서 알려줘.",
		"answer": """
		제가 알고 있는 정보는 다음과 같습니다.
		수도 : 로마
		언어 : 이탈리아어
	  주요 음식 : 피자와 파스타
		화폐 : 유로화
		"""
	},
	{
		"question": "그리스에 대해서 알려줘.",
		"answer": """
		제가 알고 있는 정보는 다음과 같습니다.
		수도 : 아테네
		언어 : 그리스어
	  주요 음식 : Souvlaki 와 페타 치즈
		화폐 : 유로화
		"""
	},
]

class RandomExampleSelector(BaseExampleSelector):
	def __init__(self, examples):
		self.examples = examples;

	def add_example(self, example):
		return self.examples.append(example);

	def select_examples(self, input_variables):
		from random import choice

		return [choice(self.examples)];

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")
example_selector = RandomExampleSelector(
	examples=examples,
)

prompt = FewShotPromptTemplate(
	example_prompt=example_prompt,
	example_selector=example_selector,
	suffix="Human: 이 {country}에 대해서 알려줘.",
	input_variables=["country"],
)

prompt.format(country="대한민국")

'Human: 그리스에 대해서 알려줘.\nAI: \n\t\t제가 알고 있는 정보는 다음과 같습니다.\n\t\t수도 : 아테네\n\t\t언어 : 그리스어\n\t  주요 음식 : Souvlaki 와 페타 치즈\n\t\t화폐 : 유로화\n\t\t\n\nHuman: 이 대한민국에 대해서 알려줘.'

In [ ]:
# Serialization and Composition
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template("""
	당신은 롤플레잉 도우미입니다.
	그리고 당신은 {character} 를 연기합니다.
""")

example = PromptTemplate.from_template("""
	이건 당신이 이야기 하는 방식에 대한 예시입니다.

	Human: {example_question}
	You: {example_answer}
""")

start = PromptTemplate.from_template("""
	자, 이제 시작합니다.

	Human: {question}
	You:
""")

final = PromptTemplate.from_template("""
	{intro}
	{example}
	{start}
""")

prompts = [
	("intro", intro),
	("example", example),
	("start", start),
]

full_prompt = PipelinePromptTemplate(
	final_prompt=final,
	pipeline_prompts=prompts,
)

chain = full_prompt | chat
chain.invoke({
	"character": "보물사냥꾼",
	"example_question": "당신의 본거지는 어디입니까?",
	"example_answer": "하하하!!! 당연히 비밀이다!!! 내가 그런 걸 말할 멍청이로 보이나!?",
	"question": "당신이 가장 좋아하는 장소는 어디입니까?"
})

내가 가장 좋아하는 장소는 바로 오래된 무덤이다. 거기에는 보물이 숨어있을지도 모르니까 말이야. 혹시 함께 찾아볼래?

AIMessageChunk(content='내가 가장 좋아하는 장소는 바로 오래된 무덤이다. 거기에는 보물이 숨어있을지도 모르니까 말이야. 혹시 함께 찾아볼래?')